# Fronius 

## Settings

In [73]:
import requests
import json
import warnings
import datetime
import dateutil
import pandas as pd

import fronius
from fronius import FroniusInverter
from fronius import FroniusArchiveJson
from fronius import FroniusRealTimeJson


from IPython.display import display, HTML, Markdown
import matplotlib.pyplot as plt
import matplotlib

inverter_ip = "192.168.1.154"
inverter_installation_time = datetime.datetime(2017, 10, 11)


In [74]:
import imp
imp.reload(fronius)

<module 'fronius' from 'C:\\Users\\tomho\\PycharmProjects\\Fronius\\fronius\\fronius.py'>

# FroniusInverter object

The first object is _FroniusInverter_.  It will act as a proxy to the inverter hardware.   Create an instance by passing the hardware's IP address.

In [55]:
host = None
fi = FroniusInverter(inverter_ip)

This python module was tested against v1 of the Fronius API, and against v1.5-4 of the firmware.  Check if the server version is tested against.

In [60]:
compatible, response = fi.check_server_compatibility()
if compatible:
    display(Markdown("**Compatibility check: OK**"))
else:
    display(Markdown("**Compatibility check: NOK**"))

print("module supports API v"+ str(fi.api_version))
print("module tested against versions", fi.tested_server_versions)

print("Fronius version response:", response)

**Compatibility check: OK**

module supports API v1
module tested against versions ['1.5-4']
Fronius version response: {'APIVersion': 1, 'BaseURL': '/solar_api/v1/', 'CompatibilityRange': '1.5-4'}


# Reading real time data from the Fronius inverter

In [76]:
json = fi.getInverterRealTimeData()
json

{'Body': {'Data': {'DAY_ENERGY': {'Unit': 'Wh', 'Values': {'1': 144}},
   'PAC': {'Unit': 'W', 'Values': {'1': 266}},
   'TOTAL_ENERGY': {'Unit': 'Wh', 'Values': {'1': 193532}},
   'YEAR_ENERGY': {'Unit': 'Wh', 'Values': {'1': 193532}}}},
 'Head': {'RequestArguments': {'DeviceClass': 'Inverter', 'Scope': 'System'},
  'Status': {'Code': 0, 'Reason': '', 'UserMessage': ''},
  'Timestamp': '2017-10-29T09:35:21+01:00'}}

use the FroniusRealTimeJson class to access the json response

In [86]:
frt = FroniusRealTimeJson(json)

if (frt.error_code() == 0):
    display(Markdown("**Realtime data call: OK**"))
    rtd = frt.data("TS")
    display(rtd)
else:
    display(Markdown("**Realtime data call: NOK**"))
    print(frt.error_status())


**Realtime data call: OK**

,TS,DAY_ENERGY,PAC,TOTAL_ENERGY,YEAR_ENERGY
0,2017-10-29 09:35:21+01:00,144,266,193532,193532


the data is marshalled in a pandas dataframe.

In [104]:
display(Markdown("**Fetch Real Time data**"))

json2 = fi.getInverterRealTimeData()
frt2 = FroniusRealTimeJson(json2)
rtd2 = frt2.data("TS")
display(rtd2)

display(Markdown("**Example: Append to previous observations**"))
rtd = pd.merge(rtd, rtd2, how='outer')
display(rtd)

display(Markdown("**Example: Perform some calculations**"))
display(rtd['DAY_ENERGY']/rtd['TOTAL_ENERGY'])


**Fetch Real Time data**

,TS,DAY_ENERGY,PAC,TOTAL_ENERGY,YEAR_ENERGY
0,2017-10-29 09:53:17+01:00,580,1900,193968,193967


**Example: Append to previous observations**

,TS,DAY_ENERGY,PAC,TOTAL_ENERGY,YEAR_ENERGY
0,2017-10-29 09:35:21+01:00,144,266,193532,193532
1,2017-10-29 09:48:27+01:00,408,1740,193795,193795
2,2017-10-29 09:48:38+01:00,415,2158,193802,193802
3,2017-10-29 09:49:04+01:00,430,2230,193817,193817
4,2017-10-29 09:50:36+01:00,486,2182,193873,193873
5,2017-10-29 09:50:56+01:00,498,2021,193885,193885
6,2017-10-29 09:53:17+01:00,580,1900,193968,193967


**Example: Perform some calculations**

0    0.000744
1    0.002105
2    0.002141
3    0.002219
4    0.002507
5    0.002569
6    0.002990
dtype: float64

In [6]:
fi = FroniusInverter(inverter_ip)
fi.findEarliestData()

http://192.168.1.154/solar_api/v1/GetArchiveData.cgi 2017-09-01 00:00:00 -> 2017-09-15 00:00:00
http://192.168.1.154/solar_api/v1/GetArchiveData.cgi 2017-09-15 00:00:00 -> 2017-09-29 00:00:00
http://192.168.1.154/solar_api/v1/GetArchiveData.cgi 2017-09-29 00:00:00 -> 2017-10-13 00:00:00


datetime.datetime(2017, 10, 11, 15, 5, tzinfo=tzoffset(None, 7200))

In [ ]:
fi = FroniusInverter(inverter_ip)
channels = fi.getChannels()

display(Markdown("**Defined channels**"))
display(channels)

In [ ]:
fi = FroniusInverter(inverter_ip)
data = fi.getHistoricalData(datetime.datetime.now() - datetime.timedelta(hours=1), datetime.datetime.now(), channels)
for key, value in data.items():
    display(Markdown("**"+str(key)+"**"))
    display(value)
    display(Markdown("\n"))



In [ ]:
fi = FroniusInverter(inverter_ip)
data = fi.getHistoricalDataJson(datetime.datetime.now() - datetime.timedelta(hours=1), datetime.datetime.now(), 
                            ["Digital_PowerManagementRelay_Out_1", "TimeSpanInSec"])

data


## Fronius configuration

explores which channels are reported, and which channels remain empty

In [ ]:
# setup the inverter object
fi = FroniusInverter(inverter_ip)

# fetch a a couple of days of recent data
all_channels = list(fi.getChannels())
jsondata=fi.getHistoricalDataJson(datetime.datetime.now() - datetime.timedelta(days=1), datetime.datetime.now(), all_channels)

# pass the 
faj = FroniusArchiveJson(jsondata)
data = faj.data()
devices = faj.device_ids()

channelsReported = []
for device in data:
    df=data[device]
    columns=list(df.columns)
    display(Markdown("**"+str(device)+"**"))
    for c in sorted(columns[1:]):
        print("\t", c)            
    channelsReported +=columns[1:]

missingChannels=all_channels
for c in channelsReported:
    missingChannels.remove(c)

display(Markdown("**"+str("Missing :")+"**"))
for c in sorted(missingChannels):
        print("\t", c) 

# daily graphs

In [ ]:
# setup the inverter object
fi = FroniusInverter(inverter_ip)

channels = ["Current_DC_String_1", "Current_DC_String_2"]
channels = [ "Current_DC_String_2"]

# fetch a a couple of days of recent data
day=datetime.datetime(year=2017, month=10, day=1)
duration = 30

data20171020 = fi.getHistoricalData(day, day + datetime.timedelta(days=duration) -datetime.timedelta(seconds=1))


In [ ]:
data20171020_json = fi.getHistoricalDataJson(day, day + datetime.timedelta(days=duration) -datetime.timedelta(seconds=1))
data20171020_json

In [ ]:
data20171020_json

In [ ]:
inverterData = data20171020['inverter/1']
inverterData

In [ ]:
fig, ax = plt.subplots()

d=inverterData
t=d["ts"]
s=d["Current_AC_Phase_1"]
ax.plot(t, s)
#ax.plot(t, t)

ax.set(xlabel='time (s)', ylabel='Current AC(A)',
       title='Current_AC_Phase_1')
ax.grid()
plt.gcf().autofmt_xdate()

plt.show()

In [ ]:
fig, ax = plt.subplots()

d=inverterData
#d=(d.sort_values('ts'))
t=d["ts"]
s1=d["Current_DC_String_1"]
ax.plot(t, s1)

s2=d["Current_DC_String_2"]
ax.plot(t, s2)

ax.set(xlabel='time (s)', ylabel='Current DC(A)',
       title='Current_DC_String')
ax.grid()
plt.gcf().autofmt_xdate()


plt.show()

In [ ]:
fig, ax = plt.subplots()

d=inverterData
#d=(d.sort_values('ts'))
t=d["ts"]
s=d["TimeSpanInSec"]
ax.plot(t, s)

ax.set(xlabel='time (s)', ylabel='Current AC(A)',
       title='TimeSpanInSec')
ax.grid()
plt.gcf().autofmt_xdate()


plt.show()

In [ ]:
fig, ax = plt.subplots()

d=inverterData
#d=(d.sort_values('ts'))
t=d["ts"]
s1=d["Current_DC_String_1"]
s2=d["Current_DC_String_2"]

ax.scatter(s1, s2, alpha=.2)

ax.set(xlabel='Current DC(A)', ylabel='Current DC(A)',
       title='Current DC 1 vs 2')
ax.grid()
plt.gcf().autofmt_xdate()


plt.show()

In [ ]:
fig, ax = plt.subplots()

d=inverterData
#d=(d.sort_values('ts'))
d['tod'] = d.ts.apply(lambda t : t.hour)

d=d[d.tod == 10]

s1=d["Current_DC_String_1"]
s2=d["Current_DC_String_2"]

ax.scatter(s1, s2, alpha=.2)

ax.set(xlabel='Current DC(A)', ylabel='Current DC(A)',
       title='Current DC 1 vs 2')
ax.grid()
plt.gcf().autofmt_xdate()


plt.show()

In [ ]:
fig, ax = plt.subplots()

d=inverterData
#d=(d.sort_values('ts'))
t=d["ts"]
s1=d["Voltage_DC_String_1"]
s2=d["Voltage_DC_String_2"]

ax.scatter(s1, s2, alpha=.2)

ax.set(xlabel='Voltage DC(A)', ylabel='Voltage DC(A)',
       title='Voltage DC 1 vs 2')
ax.grid()
plt.gcf().autofmt_xdate()


plt.show()

In [ ]:
fig, ax = plt.subplots()

d=inverterData
#d=(d.sort_values('ts'))
d['Current_DC'] = d["Current_DC_String_1"] + d["Current_DC_String_2"]

d=d[d.tod == 14]

s1=d["Current_DC_String_1"]
s2=d["Current_DC_String_2"]

ax.scatter(s1, s2, alpha=.2)

ax.set(xlabel='Current DC(A)', ylabel='Current DC(A)',
       title='Current DC 1 vs 2')
ax.grid()
plt.gcf().autofmt_xdate()


plt.show()